In [3]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [7]:
import jsonlines
import json
import os

In [8]:
i=0
with jsonlines.open('../shulexv1/dev.jsonl') as reader:
    for obj in reader:
        i+=1
        print(obj)
        print(' '.join(obj['tokens']))
        if i>5:
            break

{'id': 95709, 'text': 'i bought this specifically for a 5 day vacation to colorado .', 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'B-场景-Purpose', 'I-场景-Purpose', 'I-场景-Purpose', 'I-场景-Purpose', 'I-场景-Purpose', 'O'], 'tokens': ['i', 'bought', 'this', 'specifically', 'for', 'a', '5', 'day', 'vacation', 'to', 'colorado', '.']}
i bought this specifically for a 5 day vacation to colorado .
{'id': 96351, 'text': 'i bought this for home initially , and it worked so well that i bought it for work too .', 'tags': ['O', 'O', 'O', 'O', 'B-场景-Purpose', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-场景-Purpose', 'O', 'O'], 'tokens': ['i', 'bought', 'this', 'for', 'home', 'initially', ',', 'and', 'it', 'worked', 'so', 'well', 'that', 'i', 'bought', 'it', 'for', 'work', 'too', '.']}
i bought this for home initially , and it worked so well that i bought it for work too .
{'id': 95654, 'text': 'works great !', 'tags': ['O', 'O', 'O'], 'tokens': ['works', 'great', '!']}
works great !
{'id': 

# str idx

In [10]:
# zh2en={'B-场景-Where':'Where', 'B-场景-Purpose':'Purpose', 'B-场景-Moment':'Moment', 'B-场景-Who':'Who'}
# zh2en={'B-场景-Where':'Location of use', 'B-场景-Purpose':'Purchase Intention', 'B-场景-Moment':'Time of use', 'B-场景-Who':'Target Consumer'}
# zh2en={'B-场景-Where':'Location of use', 'B-场景-Purpose':'Reason for purchase', 'B-场景-Moment':'Usage time', 'B-场景-Who':'User'}#v4
# zh2en={'B-场景-Where':'Location of use', 'B-场景-Purpose':'Reason for purchase', 'B-场景-Moment':'Usage Moment', 'B-场景-Who':'Buy for'}#v5
# zh2en={'B-场景-Where':'Location of use', 'B-场景-Purpose':'Reason for purchase', 'B-场景-Moment':'Moment', 'B-场景-Who':'Who'}#v6
zh2en={'B-场景-Where':'Location of use', 'B-场景-Purpose':'Reason for purchase', 'B-场景-Moment':'Moment', 'B-场景-Who':'Buy for'}#v7
en2sym={'Location of use':'LOU', 'Purchase Intention':'PI', 'Time of use':'TOU', 'Target Consumer':'TC'}
RES=[]
num_dict={'B-场景-Where':0, 'B-场景-Purpose':0, 'B-场景-Moment':0, 'B-场景-Who':0}
ii=0
inp='train'
data_path='shulexv7'
with jsonlines.open('../shulexv1/{}.jsonl'.format(inp)) as reader:
    if not os.path.exists(data_path):
        os.mkdir(data_path)
    with open(data_path+'/{}.txt'.format(inp),'w') as f:
        for obj in reader:
            sent=''
            res={' '.join([' ' if i =='\n' else i for i in obj['tokens']]):[]}
            for i in range(len(obj['tags'])):
                sent+=obj['tokens'][i]+' '
                if obj['tags'][i] in ['B-场景-Who', 'B-场景-Moment', 'B-场景-Purpose', 'B-场景-Where']:
                    start=len(sent)-len(obj['tokens'][i])-1
                    num_dict[obj['tags'][i]]+=1
                    r={"text": obj['tokens'][i], "start": start, "end": None, 'prompt':zh2en[obj['tags'][i]]}
                    i+=1
                    while i<len(obj['tokens']) and obj['tags'][i] in ['I-场景-Who', 'I-场景-Moment', 'I-场景-Purpose', 'I-场景-Where']:
                        r["text"]=r["text"]+' '+obj['tokens'][i]
                        sent+=obj['tokens'][i]+' '
                        i+=1
                    r['end']=len(sent)-1
                    res[' '.join([' ' if i =='\n' else i for i in obj['tokens']])].append(r)
                    f.write(json.dumps({"content": ' '.join([' ' if i =='\n' else i for i in obj['tokens']]), "result_list": [{"text": r['text'], "start": r['start'], "end": r['end']}], "prompt": r['prompt']}))
#                     f.write('{"content": {}, "result_list": [{"text": {}, "start": {}, "end": {}}], "prompt": {}}'.format(' '.join([' ' if i =='\n' else i for i in obj['tokens']]),r['text'],r['start'],r['end'],r['prompt']))
                    f.write('\n')
            RES.append(res)
#             ii+=1
#             if ii==2:
#                 break

                
RES[0]

{'in his car ( 2007 ) fast charging .': [{'text': 'car',
   'start': 7,
   'end': 10,
   'prompt': 'Location of use'}]}

In [11]:
num_dict

{'B-场景-Where': 176, 'B-场景-Purpose': 131, 'B-场景-Moment': 396, 'B-场景-Who': 104}